In [1]:
# Importing all vision-related functions, classes, and utilities from fastai
# This gives us easy access to tools for loading, training, and evaluating deep learning models for image data.
from fastai.vision.all import *

# Setting a fixed random seed to make our results reproducible
# This ensures that the model, dataset shuffling, and other random processes give the same results every time we run the code.
set_seed(42)

# Importing parallel processing tools from fastcore
# This helps speed up certain operations by running them in parallel across multiple CPU cores.
from fastcore.parallel import *

# Importing data core utilities from fastai
# These are helpful for handling datasets, data loaders, and other data pipeline-related tasks.
from fastai.data.core import *

# Importing PyTorch, the backbone library behind fastai
# We'll use PyTorch for tensor operations, GPU acceleration, and defining deep learning models.
import torch

# Importing the time module to measure execution time of code blocks
# Useful for tracking how long training or data processing steps take.
import time


In [2]:
import pandas as pd
from pathlib import Path

train_metadata_path = Path('/Users/Yashwanth/isic/train-metadata.csv')

#  Delete all exciting files so there will be no overlaping  


In [ ]:
# Path to the working directory
working_dir = 'C:/Users/Yashwanth/ISIC24_Skin_Cancer_Detection'

# Loop through all files and folders in the directory
for filename in os.listdir(working_dir):
    file_path = os.path.join(working_dir, filename)
    try:
        # Check if it's a file or a folder
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Delete the file
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Delete the folder and its contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files and folders in the working directory have been deleted.")

Failed to delete C:/Users/Yashwanth/ISIC24_Skin_Cancer_Detection\.git. Reason: [WinError 5] Access is denied: 'C:/Users/Yashwanth/ISIC24_Skin_Cancer_Detection\\.git\\objects\\02\\77b99e76736daccaf050c824a843bf85d2aa1d'
Failed to delete C:/Users/Yashwanth/ISIC24_Skin_Cancer_Detection\Fastai. Reason: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/Yashwanth/ISIC24_Skin_Cancer_Detection\\Fastai'
All files and folders in the working directory have been deleted.
